# Realtime Flight tracking with OpenSky API, OSM and Python Bokeh

* https://opensky-network.org/
* https://bokeh.pydata.org/en/latest/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib inline sets the backend of matplotlib to the 'inline' backend:
# With this backend, the output of plotting commands is displayed inline within frontends 
# like the Jupyter notebook, directly below the code cell that produced it. 
# The resulting plots will then also be stored in the notebook document.

from opensky_api import OpenSkyApi

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.tile_providers import STAMEN_TONER
from bokeh.models import HoverTool,LinearColorMapper,LabelSet
from bokeh.palettes import RdYlBu11 as palette

##  Getting Flight Data and Convert to Pandas Data frame

To get the information about the flight, we will use the API of the Open Sky Network. We will get the data for now : t 
We will just get data of aircrat on France for that, we need to specify a range coordinate (bounding box) that consist of minimum and maximum coordinate in with latitude and longitude in geographic coordinate system (WGS84/EPSG:4326)

In [2]:
#bboxFrance= [min_latitude, max_latitude, min_longitude, max_latitude]
bboxFrance= [41, 52, -5.5, 10]

from datetime import datetime, timedelta
import time
t = int(time.time())
print(t)
print(datetime.fromtimestamp(t).strftime("%d/%m/%Y %H:%M"))

1557779284
13/05/2019 22:28


In [3]:
#  fly data
data =[]
api = OpenSkyApi()
states = api.get_states(time_secs= t, bbox=bboxFrance)
for s in states.states:
    #print("(%r, %r, %r, %r)" % (s.longitude, s.latitude, s.velocity, s.callsign))
    #print(s)
    if s.latitude != None and s.longitude != None:
       data.append((float(s.latitude), float(s.longitude), s.callsign, s.origin_country, s.baro_altitude, s.icao24))
    else :
        pass

df = pd.DataFrame(data, columns=['Lat','Long','Callsign','From','Altitude','Icao24'])
flight_df=df.fillna('No Data') #replace NAN with No Data
flight_df.head()

,Lat,Long,Callsign,From,Altitude,Icao24
0,47.4532,8.5602,SWR149D,Switzerland,No Data,4b1804
1,46.1835,6.0288,SWR37G,Switzerland,708.66,4b1805
2,47.4533,8.5594,SWR80KJ,Switzerland,281.94,4b1807
3,47.2396,8.6169,SWR63J,Switzerland,1889.76,4b1808
4,41.5399,2.7826,SWR218Q,Switzerland,12192,4b1809


In [4]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 6 columns):
Lat         765 non-null float64
Long        765 non-null float64
Callsign    765 non-null object
From        765 non-null object
Altitude    765 non-null object
Icao24      765 non-null object
dtypes: float64(2), object(4)
memory usage: 35.9+ KB


## Airlines data

https://openflights.org/data.html

In [5]:
df_airlines = pd.read_csv("data/airlines.dat", names=['id','name','alias','iata','icao','callsign','country','active'])
df_airlines.head(3)

,id,name,alias,iata,icao,callsign,country,active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N


## Aircraft database
https://opensky-network.org/datasets/metadata/
Will give the model of the Aircrat

In [6]:
df_aircraft = pd.read_csv("data/aircraftDatabase.csv", low_memory=False)
df_aircraft.head(3)

,icao24,registration,manufacturericao,manufacturername,model,typecode,serialnumber,linenumber,icaoaircrafttype,operator,...,status,built,firstflightdate,seatconfiguration,engines,modes,adsb,acars,notes,categoryDescription
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN
1,aa3487,N757F,NaN,Raytheon Aircraft Company,A36,NaN,E-3121,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN
2,ae267b,6533,VOUGHT,Aerospatiale,MH-65C Dolphin,AS65,6182,NaN,H2T,NaN,...,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,No ADS-B Emitter Category Information


We add Airlines name and Aircraft Model to our dataframe

In [7]:
flight_df['Airlines_name'] = flight_df['Callsign'].apply(lambda x: x[0:3])
flight_df["Aircraft_model"] = flight_df['Icao24']
a = df_airlines.set_index('icao')['name']
flight_df["Airlines_name"] = flight_df["Airlines_name"].replace(a)

b = df_aircraft.set_index('icao24')['typecode']
flight_df["Aircraft_model"] = flight_df["Aircraft_model"].replace(b)

flight_df.head()

,Lat,Long,Callsign,From,Altitude,Icao24,Airlines_name,Aircraft_model
0,47.4532,8.5602,SWR149D,Switzerland,No Data,4b1804,Swissair,NaN
1,46.1835,6.0288,SWR37G,Switzerland,708.66,4b1805,Swissair,NaN
2,47.4533,8.5594,SWR80KJ,Switzerland,281.94,4b1807,Swissair,NaN
3,47.2396,8.6169,SWR63J,Switzerland,1889.76,4b1808,Swissair,NaN
4,41.5399,2.7826,SWR218Q,Switzerland,12192,4b1809,Swissair,NaN


## Plotting Aircraft position on the map

We just want to view the France country when opening the page. For that, we need to specify a range coordinate (bounding box) that consist of minimum and maximum coordinate in **web Mercator coordinate system (EPSG: 3857)**. Then setup the figure and define some properties like x_range, y_range, both x and y axis type, plot height and sizing mode. Lastly by using show class, it will open a temporary html page. Using the tools on the right side of the figure you can interact with the map to pan, zoom in, zoom out, and reset the view.

Open Sky API give us the position of the aircraft with **Latitude Longitude in geographic coordinate system (WGS84/EPSG:4326)** On the other hand, the map is in web mercator coordinate system. The coordinate system is not match to each other, so we can not plot the aircraft position directly on the map. The aircraft position must be converted to web mercator coordinate system. Therefore we need to make a function to convert the latitude and longitude to mercator y and x.


In [8]:
#FUNCTION TO CONVERT GCS WGS84 TO WEB MERCATOR
def wgs84_to_web_mercator(df, lon="Long", lat="Lat"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

wgs84_to_web_mercator(flight_df)

#BOKEH COLUMN DATA SOURCE
flight_source=ColumnDataSource(flight_df)

In [9]:
# SET COLOR PALETTE
color_mapper = LinearColorMapper(palette=palette)

#SET HOVER
my_hover=HoverTool()
my_hover.tooltips=[('From','@From'),('Altitude','@Altitude'),
                   ('Airlines','@Airlines_name'),('Aircraft Model','@Aircraft_model')]
                   
#SET LABEL
labels = LabelSet(x='x', y='y', text='Callsign', level='glyph',
        x_offset=5, y_offset=5, source=flight_source, render_mode='canvas',background_fill_color='skyblue',text_font_size="8pt")

In [10]:
# DEFINE COORDINATE RANGE IN WEB MERCATOR COORDINATE SYSTEM (EPSG:3857)
#x_range,y_range=([-15187814,-6458032], [2505715,6567666])
x_range,y_range=([-2003061.5433,3890379.9687],[9478516.4071,2035957.0476])

#output_notebook()

#SETUP FIGURE
p=figure(x_range=x_range,y_range=y_range,x_axis_type='mercator',y_axis_type='mercator',
  sizing_mode='scale_width',plot_height=300)
p.add_tile(STAMEN_TONER)
p.circle('x','y',source=flight_source,fill_color="blue",size=10,fill_alpha=0.8,line_width=0.5)

#ADD HOVER TOOL AND LABELS
p.add_tools(my_hover)
p.add_layout(labels)

#SHOW FIGURE
show(p)

![title](images/screen.png)

# Visualisation interactive with folium

In [11]:
import folium
m = folium.Map(location=[48.86, 2.33])

for i in range(0,len(flight_df)):
    lat = flight_df.iloc[i]['Lat']
    lon = flight_df.iloc[i]['Long']
    callsign = flight_df.iloc[i]['Callsign']
    airlines_name = flight_df.iloc[i]['Airlines_name']
    folium.Marker([lat, lon], popup=callsign+airlines_name).add_to(m)

In [12]:
m
Ou ici de façon interactivehttps://nbviewer.jupyter.org/github/Rhizome23/aircraft-project/blob/master/RealTimeFlightTrackingBokeh.ipynb

![title](images/withfolium.png)